# ASCAD - About Side Channels Analysis and Deep Learning

[ASCAD](https://github.com/ANSSI-FR/ASCAD)

TODO make an introduction

In [2]:
!pip install numpy h5py matplotlib tqdm tensorflow

In [ ]:
!brew install python-tk

In [1]:
import os

folder_name = "ASCAD"

if not os.path.exists(folder_name):
    !git clone https://github.com/ANSSI-FR/ASCAD.git
else:
    print(f"Folder '{folder_name}' already exists. Skipping git clone.")

Folder 'ASCAD' already exists. Skipping git clone.


In [2]:
# TODO use the https://github.com/ANSSI-FR/ASCAD/blob/master/STM32_AES_v2/Readme.md
# We work on STM32 Discovery 4, so it's pretty cool if we can do that

#file_path = "ascadv2-extracted.h5"
file_path = "ASCAD_data.zip"

# Check if the file exists before downloading
if not os.path.exists(file_path):
    # !wget https://files.data.gouv.fr/anssi/ascadv2/ascadv2-extracted.h5
    !wget https://www.data.gouv.fr/s/resources/ascad/20180530-163000/ASCAD_data.zip
    !unzip ASCAD_data.zip
else:
    print(f"Folder '{file_path}' already exists. Skipping download.")

Folder 'ASCAD_data.zip' already exists. Skipping download.


In [3]:
import h5py

trained_models_file1 = "./ASCAD_data/ASCAD_trained_models/cnn_best_ascad_desync0_epochs75_classes256_batchsize200.h5"
trained_models_file2 = "./ASCAD_data/ASCAD_trained_models/cnn_best_ascad_desync100_epochs75_classes256_batchsize200.h5"
database_file1 = "./ASCAD_data/ASCAD_databases/ASCAD.h5"
database_file2 = "./ASCAD_data/ASCAD_databases/ATMega8515_raw_traces.h5"

trained_models_h5_1 = h5py.File(trained_models_file1, "r")
trained_models_h5_2 = h5py.File(trained_models_file2, "r")
database_file_h5_1 = h5py.File(database_file1, "r")
database_file_h5_2 = h5py.File(database_file2, "r")

print([k for k in trained_models_h5_1.keys()])
print([k for k in trained_models_h5_2.keys()])
print([k for k in database_file_h5_1.keys()])
print([k for k in database_file_h5_2.keys()])

['model_weights', 'optimizer_weights']
['model_weights', 'optimizer_weights']
['Attack_traces', 'Profiling_traces']
['metadata', 'traces']


In [4]:
# We need to create a file that contain a python dict for the training and testing. (Check github for more information on the keys)
import json

# We need to create a file to store the model (e.g. newModelTest.h5)
# More information on the keys: https://github.com/ANSSI-FR/ASCAD/tree/master?tab=readme-ov-file#training-the-models
training_parameters: dict = {"ascad_database": "ASCAD_data/ASCAD_databases/ASCAD.h5",
                             "training_model": "newModelTest.keras",
                             "network_type": "mlp",
                             "epochs": 800,
                             "batch_size": 100}

# More information on the keys: https://github.com/ANSSI-FR/ASCAD/tree/master?tab=readme-ov-file#testing-the-trained-models
testing_parameters_mld: dict = {"model_file": "ASCAD_data/ASCAD_trained_models/mlp_best_ascad_desync0_node200_layernb6_epochs200_classes256_batchsize100.h5",
                            "ascad_database": "ASCAD_data/ASCAD_databases/ASCAD.h5",
                            "num_traces": 2000,
                            "save_file": "plot_test_model_mld_desync0.png"}

testing_parameters_cnn: dict = {"model_file": "ASCAD_data/ASCAD_trained_models/cnn_best_ascad_desync0_epochs75_classes256_batchsize200.h5",
                            "ascad_database": "ASCAD_data/ASCAD_databases/ASCAD.h5",
                            "num_traces": 2000,
                            "save_file": "plot_test_model_cnn_desync0.png"}


with open('training_parameters.txt', 'w') as training_file:
     training_file.write(json.dumps(training_parameters))

with open('testing_parameters_mld.txt', 'w') as testing_file:
     testing_file.write(json.dumps(testing_parameters_mld))

with open('testing_parameters_cnn.txt', 'w') as testing_file:
     testing_file.write(json.dumps(testing_parameters_cnn))

# TODO fix ASCAD_train_models, seems there're a lot of issue with the new version of keras, need to solve all of them

In [5]:
!python3 ASCAD/ASCAD_train_models.py training_parameters.txt

/Users/gaetan/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Traceback (most recent call last):
  File "/Users/gaetan/Documents/M1/ADL/s2/M1_S2_ADL/ASCAD/ASCAD/ASCAD_train_models.py", line 397, in <module>
    train_model(X_profiling, Y_profiling, best_model, training_model, epochs, batch_size, multilabel, validation_split, early_stopping)
  File "/Users/gaetan/Documents/M1/ADL/s2/M1_S2_ADL/ASCAD/ASCAD/ASCAD_train_models.py", line 294, in train_model
    if isinstance(model.get_layer(index=0).input_shape, list):
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Dense' object has no attribute 'input_shape'. Did you mean: 'input_dtype'?


In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
%run ASCAD/ASCAD_test_models.py testing_parameters_mld.txt

ModuleNotFoundError: No module named '_tkinter'

In [8]:
# It's time to test the model we have created (only change with the ASCAD_trained_models is the number of epochs 800 compare to 200)
# According to the paper, the model should perform better
testing_parameters_new_model: dict = {"model_file": "newModelTest.h5",
                            "ascad_database": "ASCAD_data/ASCAD_databases/ASCAD.h5",
                            "num_traces": 1000,
                            "save_file": "plot_new_model.png"}

with open('testing_parameters_new_model.txt', 'w') as training_file:
     training_file.write(json.dumps(testing_parameters_new_model))

In [9]:
# Modify `savefig`, add `bbox_inches='tight'` so the image is not cropped (plt.savefig(save_file, bbox_inches='tight'))
%run ASCAD/ASCAD_test_models.py testing_parameters_new_model.txt

ModuleNotFoundError: No module named '_tkinter'